### Import and webdriver setup

In [116]:
from time import sleep

In [147]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('C:/Users/jackie.yu/Documents/WeCloudData/Python/PlayStoreProject/chromedriver.exe')
driver.get("https://play.google.com/store/apps/details?id=cc.forestapp&showAllReviews=true")

### Parsing through the html with Selenium

In [148]:
# Expands the "Read more" button for reviews and scrolls
for _ in range(10):   
    button_list = driver.find_elements_by_css_selector("button[jsname='gxjVle']")
    for button in button_list:
        driver.execute_script("arguments[0].click();", button)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)
    try: 
        driver.find_element_by_class_name("U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ.M9Bg4d").click()
    
        print("i clicked")
    except:
        print("me pass")
        pass

me pass
me pass
me pass
i clicked
me pass
me pass
i clicked
me pass
me pass
i clicked


In [149]:
# Grabs the non-expandable reviews and puts them into a dictionary with an index
# Expandable reviews will show up as an empty space in the dictionary due to "display = none" on the css
list_of_reviews = driver.find_elements_by_css_selector("span[jsname='bN97Pc']")
dict1 = {}
for i, review in enumerate(list_of_reviews):
    dict1[i] = review.text
dict1

{0: '',
 1: '',
 2: '',
 3: '',
 4: 'So much helpful.... A great app to deal with phone addiction. Now I am a premium user. Definitely, the developers are working hard. This is by far the most productive app on play store and I am increasing my productivity day by day with you guys. Looking forward to plant a real 🎄! Great job guys!',
 5: '',
 6: 'This app is amazing. However, the devs have in the past removed features for free users in favour of making them paid. I currently am using the paid version, but for anyone using it free, just be aware of this. (Referring to the loss of app whitelisting in the free version)',
 7: '',
 8: '',
 9: '',
 10: '',
 11: "It's an ok app, but you can't play the ambient sounds while planting the tree, I had a sound unlocked but I could never play it while I studied, and some other advanced settings aren't easy to figure out either. Also, it would be nice to have at least a few achievements in the free version.",
 12: '',
 13: '',
 14: 'I meet a problem

In [155]:
%%timeit
# Grabs the expandable reviews and puts them into a dictionary with an index
# Non-Expandable reviews will show up as an empty space in the dictionary due to "display = none" on the css
list_of_reviews2 = driver.find_elements_by_css_selector("span[jsname='fbQN7e']")
dict2 = {}
for i, review in enumerate(list_of_reviews2):
    dict2[i] = review.text
dict2

5.34 s ± 79.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
# Due to the nature of the dictionaries, we can replace the empty space with values from the opposite dictionary
# to make the full list of the review
# index is required to match with other data like stars and date
for i in dict1:
    if dict1[i] == '':
        dict1[i] = dict2[i]
dict1

{0: "I can't put into words how much this app provides my needs and actions. By upgrading to plus, I was able to plant real trees just in the comfort of my home! It is indeed worth it. It is affordable and sustainable as well. For the creators, great job! I have been using this app for 3 years and it hasn't failed me since. Suggestion though, create an extension that works like momentum, so the timer in my chrome won't go away when I click. Thanks!",
 1: "Super cute and helpful! This has been helping me with my studying since I started using it - a lot of the time, I'll even go over the time I set for myself. I love seeing my little trees grow because I'm working hard! Having the reminders also helps to get me back into study mode. I really love this app!",
 2: 'Nice app but the only thing irritates me is premiership. Any time i want explore the app its asks me to take premium orelse it wont open. I mean you can only use the timer nothing else without premiership. If i have to use only